In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
cd ../follidet/

In [ ]:
# import sys
# sys.path.append('../yolov5/')

In [ ]:
import os
import cv2
import glob
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import Counter
from matplotlib.patches import Rectangle

In [ ]:
from params import *
from inference import predict
from model import define_model

from data.dataset import InferenceDataset
from data.transforms import OCT_preprocess

from util.plot import plot_sample, plot_predictions
from util.torch_utils import load_model_weights, seed_everything
from util.saving import load_stack
from util.boxes import yolo_to_volumes, expand_boxes

from util.logger import Config

## Data

In [ ]:
DATA_PATH = "../../../datasets/lipotec/"
SEG_PATH = "../../../datasets/segmentations/lipotec_2/"
NAME = "whitening"

In [ ]:
STEP = 5
PLOT = False
SAVE = True

SAVE_PATH = OUT_DIR

In [ ]:
stack_paths = {}
for root, dirs, files in os.walk(DATA_PATH):
    for filename in files:
        if filename.lower().endswith(".dcm") or filename.lower().endswith(".tiff"):
            if "macro-proc" not in filename:
                stack_paths[filename.split('.')[0]]= (os.path.join(root, filename))

print(f'Found {len(stack_paths)} stacks')

In [ ]:
for stack in stack_paths:
    seg_path = SEG_PATH + f"{stack}_contour.tif"
    
    assert os.path.exists(seg_path)

## Model

In [ ]:
EXP_FOLDER = LOG_PATH + "2021-07-12/1/"  # yolov5s
EXP_FOLDER = LOG_PATH + "2021-07-12/0/"  # d1

In [ ]:
config = Config(json.load(open(EXP_FOLDER + "config.json", 'r')))

In [ ]:
FOLD_IDX = 0

In [ ]:
weights = sorted([EXP_FOLDER + p for p in os.listdir(EXP_FOLDER) if 'pt' in p])

In [ ]:
model = define_model(
    config
).to(config.device)
model.zero_grad()

model.eval()

model = load_model_weights(model, weights[FOLD_IDX])

## Data

In [ ]:
%matplotlib inline

transforms = OCT_preprocess(augment=False, visualize=False)
volumes_dict = {}

for path in tqdm(stack_paths):
    print(f' -> Stack {path}')

    stack = load_stack(stack_paths[path])
    if len(stack.shape) == 2:
        continue

    dataset = InferenceDataset(
        stack_paths[path],
        SEG_PATH + f"{path}_contour.tif",
        step=5,
        transforms=transforms
    )
    
    meter = predict(model, dataset, config)
    
    meter.pred_fusion = [expand_boxes(box, 1.2) for box in meter.pred_fusion]

    meter.plot(n_samples=-1, fused=True)
    
    boxes = meter.pred_fusion[0]
    volumes = yolo_to_volumes(boxes["yolo"], stack.shape[0], stack.shape[-1])
    volumes_dict[path] = volumes
    
#     break

json.dump(volumes_dict, open(f"../output/volumes_{NAME}.json", "w"))